In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

* We support Llama, Mistral, CodeLlama, TinyLlama, Vicuna, Open Hermes etc
* And Yi, Qwen ([llamafied](https://huggingface.co/models?sort=trending&search=qwen+llama)), Deepseek, all Llama, Mistral derived archs.
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* [**NEW**] With [PR 26037](https://github.com/huggingface/transformers/pull/26037), we support downloading 4bit models **4x faster**! [Our repo](https://huggingface.co/unsloth) has Llama, Mistral 4bit models.

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/codellama-7b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/3.87G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/539 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `ChatML` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [4]:
from datasets import load_dataset
dataset = load_dataset("AlfredPros/smart-contracts-instructions", split="train")

Generating train split:   0%|          | 0/6003 [00:00<?, ? examples/s]

In [5]:
dataset

Dataset({
    features: ['instruction', 'source_code', 'model'],
    num_rows: 6003
})

In [6]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs       = examples["source_code"]

    texts = []
    for instruction, output in zip(instructions, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [7]:

# from datasets import load_dataset
# dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/6003 [00:00<?, ? examples/s]

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 10,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/6003 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [9]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 6,003 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 10
 "-____-"     Number of trainable parameters = 39,976,960


Step,Training Loss
1,0.385800
2,0.630200
3,0.429500
4,0.519300
5,0.360500
6,0.329700
7,0.454900
8,0.343500
9,0.286600
10,0.365800


In [10]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Make a smart contract to manage a token called ZildFinanceCoin. The contract allows users to transfer tokens, burn tokens, and approve token transfers. It also includes functionality to change the founder address, set a minter address, set a furnace address, and freeze/unfreeze user accounts.", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 5000, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nMake a smart contract to manage a token called ZildFinanceCoin. The contract allows users to transfer tokens, burn tokens, and approve token transfers. It also includes functionality to change the founder address, set a minter address, set a furnace address, and freeze/unfreeze user accounts.\n\n### Response:\npragma solidity ^0.5.0;\n\ncontract ZildFinanceCoin {\n    string public name = "ZildFinanceCoin";\n    string public symbol = "ZIL";\n    uint8 public decimals = 18;\n    uint256 public totalSupply;\n    address public founder;\n    address public minter;\n    address public furnace;\n    mapping(address => uint256) public balances;\n    mapping(address => mapping(address => uint256)) public allowance;\n    mapping(address => bool) public frozen;\n\n    event Transfer(address indexed from, address

* We support Llama, Mistral, CodeLlama, TinyLlama, Vicuna, Open Hermes etc
* And Yi, Qwen ([llamafied](https://huggingface.co/models?sort=trending&search=qwen+llama)), Deepseek, all Llama, Mistral derived archs.
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* [**NEW**] With [PR 26037](https://github.com/huggingface/transformers/pull/26037), we support downloading 4bit models **4x faster**! [Our repo](https://huggingface.co/unsloth) has Llama, Mistral 4bit models.

In [11]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit", # Instruct version of Gemma 7b
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit", # Instruct version of Gemma 2b
    "unsloth/llama-3-8b-bnb-4bit", # [NEW] 15 Trillion token Llama-3
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [12]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `ChatML` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [13]:
from nltk.translate.bleu_score import corpus_bleu
from datasets import load_dataset

# Load your generated responses and corresponding references
# For this example, let's assume you have stored your generated responses in a list called `generated_responses`
# and references from the Alpaca dataset in a list called `reference_responses`
# You can replace these lists with your actual data
generated_responses = [
    """pragma solidity ^0.8.0;

contract ZildFinanceCoin {
    address public founder;
    address public minter;
    address public furnace;
    mapping(address => uint256) public balances;
    mapping(address => mapping(address => uint256)) public allowed;

    event Transfer(address indexed from, address indexed to, uint256 value);
    event Approval(address indexed owner, address indexed spender, uint256 value);
    event Burn(address indexed from, uint256 value);
    event Freeze(address indexed account, bool freeze);
    event ChangeFounder(address indexed founder);
    event ChangeMinter(address indexed minter);
    event ChangeFurnace(address indexed furnace);

    constructor(address _founder, address _minter, address _furnace) {
        founder = _founder;
        minter = _minter;
        furnace = _furnace;
    }

    function transfer(address recipient, uint256 amount) external {
        _transfer(msg.sender, recipient, amount);
    }

    function approve(address spender, uint256 amount) external {
        allowed[msg.sender][spender] = amount;
        emit Approval(msg.sender, spender, amount);
    }

    function transferFrom(address sender, address recipient, uint256 amount) external {
        _transfer(sender, recipient, amount);
        _approve(sender, msg.sender, allowed[sender][msg.sender] - amount);
    }

    function burn(uint256 amount) external {
        _burn(msg.sender, amount);
    }

    function freezeAccount(address account, bool freeze) external {
        require(msg.sender == founder || msg.sender == minter || msg.sender == furnace, "Not authorized");
        emit Freeze(account, freeze);
    }

    function changeFounder(address _founder) external {
        require(msg.sender == founder, "Not authorized");
        founder = _founder;
        emit ChangeFounder(_founder);
    }

    function changeMinter(address _minter) external {
        require(msg.sender == founder, "Not authorized");
        minter = _minter;
        emit ChangeMinter(_minter);
    }

    function changeFurnace(address _furnace) external {
        require(msg.sender == founder, "Not authorized");
        furnace = _furnace;
        emit ChangeFurnace(_furnace);
    }

    function _transfer(address sender, address recipient, uint256 amount) internal {
        require(sender != address(0), "ERC20: transfer from the zero address");
        require(recipient != address(0), "ERC20: transfer to the zero address");
        require(amount > 0, "ERC20: amount must be greater than zero");

        if (balances[sender] < amount) {
            revert("ERC20: transfer amount exceeds balance");
        }

        if (balances[recipient] + amount < balances[recipient]) {
            revert("ERC20: recipient balance will overflow");
        }

        balances[sender] -= amount;
        balances[recipient] += amount;
        emit Transfer(sender, recipient, amount);
    }

    function _approve(address owner, address spender, uint256 amount) internal {
        require(owner != address(0), "ERC20: approve from the zero address");
        require(spender != address(0), "ERC20: approve to the zero address");
        require(amount > 0, "ERC20: amount must be greater than zero");

        allowed[owner][spender] = amount;
        emit Approval(owner, spender, amount);
    }

    function _burn(address account, uint256 amount) internal {
        require(account != address(0), "ERC20: burn from the zero address");
        require(amount > 0, "ERC20: amount must be greater than zero");

        if (balances[account] < amount) {
            revert("ERC20: amount exceeds balance");
        }

        if (balances[account] + amount < balances[account]) {
            revert("ERC20: recipient balance will overflow");
        }

        balances[account] -= amount;
        emit Burn(account, amount);
    }
}
"""
]

# Load the reference dataset
reference_dataset = load_dataset("yahma/alpaca-cleaned", split="train")
reference_responses = reference_dataset["output"]

# Calculate BLEU score
references = [[response] for response in reference_responses]
bleu_score = corpus_bleu(references, generated_responses)

print("BLEU Score:", bleu_score)


Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

AssertionError: The number of hypotheses and their reference(s) should be the same 

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

# Generated code
generated_code = """\npragma solidity ^0.8.0;\n\ncontract ZildFinanceCoin {\n    address public founder;\n    address public minter;\n    address public furnace;\n    mapping(address => uint256) public balances;\n    mapping(address => mapping(address => uint256)) public allowed;\n    event Transfer(address indexed from, address indexed to, uint256 value);\n    event Approval(address indexed owner, address indexed spender, uint256 value);\n    event Burn(address indexed from, uint256 value);\n    event Freeze(address indexed account, bool freeze);\n    event ChangeFounder(address indexed founder);\n    event ChangeMinter(address indexed minter);\n    event ChangeFurnace(address indexed furnace);\n    constructor(address _founder, address _minter, address _furnace) {\n        founder = _founder;\n        minter = _minter;\n        furnace = _furnace;\n    }\n    function transfer(address recipient, uint256 amount) external {\n        _transfer(msg.sender, recipient, amount);\n    }\n    function approve(address spender, uint256 amount) external {\n        allowed[msg.sender][spender] = amount;\n        emit Approval(msg.sender, spender, amount);\n    }\n    function transferFrom(address sender, address recipient, uint256 amount) external {\n        _transfer(sender, recipient, amount);\n        _approve(sender, msg.sender, allowed[sender][msg.sender] - amount);\n    }\n    function burn(uint256 amount) external {\n        _burn(msg.sender, amount);\n    }\n    function freezeAccount(address account, bool freeze) external {\n        require(msg.sender == founder || msg.sender == minter || msg.sender == furnace, "Not authorized");\n        emit Freeze(account, freeze);\n    }\n    function changeFounder(address _founder) external {\n        require(msg.sender == founder, "Not authorized");\n        founder = _founder;\n        emit ChangeFounder(_founder);\n    }\n    function changeMinter(address _minter) external {\n        require(msg.sender == founder, "Not authorized");\n        minter = _minter;\n        emit ChangeMinter(_minter);\n    }\n    function changeFurnace(address _furnace) external {\n        require(msg.sender == founder, "Not authorized");\n        furnace = _furnace;\n        emit ChangeFurnace(_furnace);\n    }\n    function _transfer(address sender, address recipient, uint256 amount) internal {\n        require(sender!= address(0), "ERC20: transfer from the zero address");\n        require(recipient!= address(0), "ERC20: transfer to the zero address");\n        require(amount > 0, "ERC20: amount must be greater than zero");\n        if (balances[sender] < amount) {\n            revert("ERC20: transfer amount exceeds balance");\n        }\n        if (balances[recipient] + amount < balances[recipient]) {\n            revert("ERC20: recipient balance will overflow");\n        }\n        balances[sender] -= amount;\n        balances[recipient] += amount;\n        emit Transfer(sender, recipient, amount);\n    }\n    function _approve(address owner, address spender, uint256 amount) internal {\n        require(owner!= address(0), "ERC20: approve from the zero address");\n        require(spender!= address(0), "ERC20: approve to the zero address");\n        require(amount > 0, "ERC20: amount must be greater than zero");\n        allowed[owner][spender] = amount;\n        emit Approval(owner, spender, amount);\n    }\n    function _burn(address account, uint256 amount) internal {\n        require(account!= address(0), "ERC20: burn from the zero address");\n        require(amount > 0, "ERC20: amount must be greater than zero");\n        if (balances[account] < amount) {\n            revert("ERC20: amount exceeds balance");\n        }\n        if (balances[account] + amount < balances[account]) {\n            revert("ERC20: recipient balance will overflow");\n        }\n        balances[account] -= amount;\n        emit Burn(account, amount);\n  }\n"""

# Actual code
actual_code = """pragma solidity 0.5.4; interface IERC20 { function balanceOf(address account) external view returns (uint256); function transfer(address recipient, uint256 amount) external returns (bool); function allowance(address owner, address spender) external view returns (uint256); function approve(address spender, uint256 amount) external returns (bool); function transferFrom(address sender, address recipient, uint256 amount) external returns (bool); event Transfer(address indexed from, address indexed to, uint256 value); event Approval(address indexed owner, address indexed spender, uint256 value); } contract Ownable { address private _owner; event OwnershipTransferred(address indexed previousOwner, address indexed newOwner); constructor() internal { _owner = msg.sender; emit OwnershipTransferred(address(0), _owner); } function owner() public view returns (address) { return _owner; } function isOwner() public view returns (bool) { return msg.sender == _owner; } modifier onlyOwner() { require(msg.sender == _owner, "Ownable: caller is not the owner"); _; } function transferOwnership(address newOwner) public onlyOwner { require(newOwner != address(0), "Ownable: new owner is the zero address"); emit OwnershipTransferred(_owner, newOwner); _owner = newOwner; } } library SafeMath { /** * @dev Returns the addition of two unsigned integers, reverting on * overflow. * * Counterpart to Solidity's `+` operator. * * Requirements: * - Addition cannot overflow. */ function add(uint256 a, uint256 b) internal pure returns (uint256) { uint256 c = a + b; require(c >= a, "SafeMath: addition overflow"); return c; } /** * @dev Returns the subtraction of two unsigned integers, reverting on * overflow (when the result is negative). * * Counterpart to Solidity's `-` operator. * * Requirements: * - Subtraction cannot overflow. */ function sub(uint256 a, uint256 b) internal pure returns (uint256) { return sub(a, b, "SafeMath: subtraction overflow"); } /** * @dev Returns the subtraction of two unsigned integers, reverting with custom message on * overflow (when the result is negative). * * Counterpart to Solidity's `-` operator. * * Requirements: * - Subtraction cannot overflow. * * _Available since v2.4.0._ */ function sub(uint256 a, uint256 b, string memory errorMessage) internal pure returns (uint256) { require(b <= a, errorMessage); uint256 c = a - b; return c; } /** * @dev Returns the multiplication of two unsigned integers, reverting on * overflow. * * Counterpart to Solidity's `*` operator. * * Requirements: * - Multiplication cannot overflow. */ function mul(uint256 a, uint256 b) internal pure returns (uint256) { // Gas optimization: this is cheaper than requiring 'a' not being zero, but the // benefit is lost if 'b' is also tested. // See: https://github.com/OpenZeppelin/openzeppelin-contracts/pull/522 if (a == 0) { return 0; } uint256 c = a * b; require(c / a == b, "SafeMath: multiplication overflow"); return c; } /** * @dev Returns the integer division of two unsigned integers. Reverts on * division by zero. The result is rounded towards zero. * * Counterpart to Solidity's `/` operator. Note: this function uses a * `revert` opcode (which leaves remaining gas untouched) while Solidity * uses an invalid opcode to revert (consuming all remaining gas). * * Requirements: * - The divisor cannot be zero. */ function div(uint256 a, uint256 b) internal pure returns (uint256) { return div(a, b, "SafeMath: division by zero"); } /** * @dev Returns the integer division of two unsigned integers. Reverts with custom message on * division by zero. The result is rounded towards zero. * * Counterpart to Solidity's `/` operator. Note: this function uses a * `revert` opcode (which leaves remaining gas untouched) while Solidity * uses an invalid opcode to revert (consuming all remaining gas). * * Requirements: * - The divisor cannot be zero. * * _Available since v2.4.0._ */ function div(uint256 a, uint256 b, string memory errorMessage) internal pure returns (uint256) { // Solidity only automatically asserts when dividing by 0 require(b > 0, errorMessage); uint256 c = a / b; // assert(a == b * c + a % b); // There is no case in which this doesn't hold return c; } /** * @dev Returns the remainder of dividing two unsigned integers. (unsigned integer modulo), * Reverts when dividing by zero. * * Counterpart to Solidity's `%` operator. This function uses a `revert` * opcode (which leaves remaining gas untouched) while Solidity uses an * invalid opcode to revert (consuming all remaining gas). * * Requirements: * - The divisor cannot be zero. */ function mod(uint256 a, uint256 b) internal pure returns (uint256) { return mod(a, b, "SafeMath: modulo by zero"); } /** * @dev Returns the remainder of dividing two unsigned integers. (unsigned integer modulo), * Reverts with custom message when dividing by zero. * * Counterpart to Solidity's `%` operator. This function uses a `revert` * opcode (which leaves remaining gas untouched) while Solidity uses an * invalid opcode to revert (consuming all remaining gas). * * Requirements: * - The divisor cannot be zero. * * _Available since v2.4.0._ */ function mod(uint256 a, uint256 b, string memory errorMessage) internal pure returns (uint256) { require(b != 0, errorMessage); return a % b; } } contract ZildFinanceCoin is Ownable, IERC20 { using SafeMath for uint256; string public constant name = 'Zild Finance Coin'; string public constant symbol = 'Zild'; uint8 public constant decimals = 18; uint256 public totalSupply = 9980 * 10000 * 10 ** uint256(decimals); uint256 public allowBurn = 2100 * 10000 * 10 ** uint256(decimals); uint256 public tokenDestroyed; uint256 public constant FounderAllocation = 1497 * 10000 * 10 ** uint256(decimals); uint256 public constant FounderLockupAmount = 998 * 10000 * 10 ** uint256(decimals); uint256 public constant FounderLockupCliff = 365 days; uint256 public constant FounderReleaseInterval = 30 days; uint256 public constant FounderReleaseAmount = 20.7916 * 10000 * 10 ** uint256(decimals); uint256 public constant MarketingAllocation = 349 * 10000 * 10 ** uint256(decimals); uint256 public constant FurnaceAllocation = 150 * 10000 * 10 ** uint256(decimals); address public founder = address(0); uint256 public founderLockupStartTime = 0; uint256 public founderReleasedAmount = 0; mapping (address => uint256) private _balances; mapping (address => mapping (address => uint256)) private _allowances; mapping (address => bool) public frozenAccount; event Transfer(address indexed from, address indexed to, uint256 value); event Approval(address indexed from, address indexed to, uint256 value); event ChangeFounder(address indexed previousFounder, address indexed newFounder); event SetMinter(address indexed minter); event SetMarketing(address indexed marketing); event SetFurnace(address indexed furnace); event Burn(address indexed _from, uint256 _tokenDestroyed, uint256 _timestamp); event FrozenFunds(address target, bool frozen); constructor(address _founder, address _marketing) public { require(_founder != address(0), "ZildFinanceCoin: founder is the zero address"); require(_marketing != address(0), "ZildFinanceCoin: operator is the zero address"); founder = _founder; founderLockupStartTime = block.timestamp; _balances[address(this)] = totalSupply; _transfer(address(this), _marketing, MarketingAllocation); } function release() public { uint256 currentTime = block.timestamp; uint256 cliffTime = founderLockupStartTime.add(FounderLockupCliff); if (currentTime < cliffTime) return; if (founderReleasedAmount >= FounderLockupAmount) return; uint256 month = currentTime.sub(cliffTime).div(FounderReleaseInterval); uint256 releaseAmount = month.mul(FounderReleaseAmount); if (releaseAmount > FounderLockupAmount) releaseAmount = FounderLockupAmount; if (releaseAmount <= founderReleasedAmount) return; uint256 amount = releaseAmount.sub(founderReleasedAmount); founderReleasedAmount = releaseAmount; _transfer(address(this), founder, amount); } function balanceOf(address account) public view returns (uint256) { return _balances[account]; } function transfer(address to, uint256 amount) public returns (bool) { require(to != address(0), "ERC20: tranfer to the zero address"); require(!frozenAccount[msg.sender]); require(!frozenAccount[to]); _transfer(msg.sender, to, amount); return true; } function burn(uint256 _value) public returns (bool){ _burn(msg.sender, _value); return true; } function _burn(address _who, uint256 _burntAmount) internal { require (tokenDestroyed.add(_burntAmount) <= allowBurn, "ZildFinanceCoin: exceeded the maximum allowable burning amount" ); require(_balances[msg.sender] >= _burntAmount && _burntAmount > 0); _transfer(address(_who), address(0), _burntAmount); totalSupply = totalSupply.sub(_burntAmount); tokenDestroyed = tokenDestroyed.add(_burntAmount); emit Burn(_who, _burntAmount, block.timestamp); } function allowance(address from, address to) public view returns (uint256) { return _allowances[from][to]; } function approve(address to, uint256 amount) public returns (bool) { _approve(msg.sender, to, amount); return true; } function transferFrom(address from, address to, uint256 amount) public returns (bool) { uint256 remaining = _allowances[from][msg.sender].sub(amount, "ERC20: transfer amount exceeds allowance"); require(to != address(0), "ERC20: tranfer to the zero address"); require(!frozenAccount[from]); require(!frozenAccount[to]); require(!frozenAccount[msg.sender]); _transfer(from, to, amount); _approve(from, msg.sender, remaining); return true; } function _transfer(address from, address to, uint256 amount) private { require(from != address(0), "ERC20: transfer from the zero address"); _balances[from] = _balances[from].sub(amount, "ERC20: transfer amount exceeds balance"); _balances[to] = _balances[to].add(amount); emit Transfer(from, to, amount); } function _approve(address from, address to, uint256 amount) private { require(from != address(0), "ERC20: approve from the zero address"); require(to != address(0), "ERC20: approve to the zero address"); _allowances[from][to] = amount; emit Approval(from, to, amount); } function changeFounder(address _founder) public onlyOwner { require(_founder != address(0), "ZildFinanceCoin: founder is the zero address"); emit ChangeFounder(founder, _founder); founder = _founder; } function setMinter(address minter) public onlyOwner { require(minter != address(0), "ZildFinanceCoin: minter is the zero address"); require(_balances[minter] == 0, "ZildFinanceCoin: minter has been initialized"); _transfer(address(this), minter, totalSupply.sub(FounderAllocation)); emit SetMinter(minter); } function setFurnace(address furnace) public onlyOwner { require(furnace != address(0), "ZildFinanceCoin: furnace is the zero address"); require(_balances[furnace] == 0, "ZildFinanceCoin: furnace has been initialized"); _transfer(address(this), furnace, FurnaceAllocation); emit SetFurnace(furnace); } function freezeAccount(address _target, bool _bool) public onlyOwner { if (_target != address(0)) { frozenAccount[_target] = _bool; emit FrozenFunds(_target,_bool); } } }"""

# Tokenize the generated and actual code
gen_tokens = generated_code.split()
act_tokens = actual_code.split()

# Calculate BLEU score
bleu_score = sentence_bleu([act_tokens], gen_tokens)
print("BLEU Score:", bleu_score)


In [ ]:
from nltk.translate.bleu_score import sentence_bleu
generated_code = """ \npragma solidity ^0.5.0;\n\ncontract ZildFinanceCoin {\n    string public name = "ZildFinanceCoin";\n    string public symbol = "ZIL";\n    uint8 public decimals = 18;\n    uint256 public totalSupply;\n    address public founder;\n    address public minter;\n    address public furnace;\n    mapping(address => uint256) public balances;\n    mapping(address => mapping(address => uint256)) public allowance;\n    mapping(address => bool) public frozen;\n\n    event Transfer(address indexed from, address indexed to, uint256 value);\n    event Approval(address indexed owner, address indexed spender, uint256 value);\n    event MinterChanged(address indexed from, address indexed to);\n    event FurnaceChanged(address indexed from, address indexed to);\n    event FounderChanged(address indexed from, address indexed to);\n    event FrozenFunds(address indexed target, bool frozen);\n\n    constructor() public {\n        totalSupply = 1000000000000000000000000000;\n        balances[msg.sender] = totalSupply;\n        founder = msg.sender;\n        minter = msg.sender;\n        furnace = msg.sender;\n    }\n\n    modifier onlyFounder() {\n        require(msg.sender == founder, "Only founder can call this function.");\n        _;\n    }\n\n    modifier onlyMinter() {\n        require(msg.sender == minter, "Only minter can call this function.");\n        _;\n    }\n\n    modifier onlyFurnace() {\n        require(msg.sender == furnace, "Only furnace can call this function.");\n        _;\n    }\n\n    modifier onlyMinterOrFurnace() {\n        require(msg.sender == minter || msg.sender == furnace, "Only minter or furnace can call this function.");\n        _;\n    }\n\n    function transfer(address _to, uint256 _value) public returns (bool success) {\n        require(_to != address(0), "Invalid address.");\n        require(balances[msg.sender] >= _value, "Insufficient balance.");\n        require(!frozen[msg.sender], "Account frozen.");\n        balances[msg.sender] = balances[msg.sender].sub(_value);\n        balances[_to] = balances[_to].add(_value);\n        emit Transfer(msg.sender, _to, _value);\n        return true;\n    }\n\n    function approve(address _spender, uint256 _value) public returns (bool success) {\n        require(_spender != address(0), "Invalid address.");\n        allowance[msg.sender][_spender] = _value;\n        emit Approval(msg.sender, _spender, _value);\n        return true;\n    }\n\n    function transferFrom(address _from, address _to, uint256 _value) public returns (bool success) {\n        require(_to != address(0), "Invalid address.");\n        require(balances[_from] >= _value, "Insufficient balance.");\n        require(balances[_to] + _value >= balances[_to], "Overflow.");\n        require(!frozen[_from], "Account frozen.");\n        balances[_from] = balances[_from].sub(_value);\n        balances[_to] = balances[_to].add(_value);\n        allowance[_from][msg.sender] = allowance[_from][msg.sender].sub(_value);\n        emit Transfer(_from, _to, _value);\n        return true;\n    }\n\n    function burn(uint256 _value) public returns (bool success) {\n        require(balances[msg.sender] >= _value, "Insufficient balance.");\n        balances[msg.sender] = balances[msg.sender].sub(_value);\n        totalSupply = totalSupply.sub(_value);\n        emit Transfer(msg.sender, address(0), _value);\n        return true;\n    }\n\n    function changeMinter(address _newMinter) public onlyFounder returns (bool success) {\n        require(_newMinter != address(0), "Invalid address.");\n        minter = _newMinter;\n        emit MinterChanged(msg.sender, _newMinter);\n        return true;\n    }\n\n    function changeFurnace(address _newFurnace) public onlyFounder returns (bool success) {\n        require(_newFurnace != address(0), "Invalid address.");\n        furnace = _newFurnace;\n        emit FurnaceChanged(msg.sender, _newFurnace);\n        return true;\n    }\n\n    function changeFounder(address _newFounder) public onlyFounder returns (bool success) {\n        require(_newFounder != address(0), "Invalid address.");\n        founder = _newFounder;\n        emit FounderChanged(msg.sender, _newFounder);\n        return true;\n    }\n\n    function freezeAccount(address _target) public onlyFounder returns (bool success) {\n        require(_target != address(0), "Invalid address.");\n        frozen[_target] = true;\n        emit FrozenFunds(_target, true);\n        return true;\n    }\n\n    function unfreezeAccount(address _target) public onlyFounder returns (bool success) {\n        require(_target != address(0), "Invalid address.");\n        frozen[_target] = false;\n        emit FrozenFunds(_target, false);\n        return true;\n    }\n\n    function mint(address _target, uint256 _value) public onlyMinterOrFurnace returns (bool success) {\n        require(_target != address(0), "Invalid address.");\n        require(balances[_target] + _value >= balances[_target], "Overflow.");\n        balances[_target] = balances[_target].add(_value);\n        totalSupply = totalSupply.add(_value);\n        emit Transfer(address(0), _target, _value);\n        return true;\n    }\n\n    function burnFrom(address _from, uint256 _value) public onlyMinterOrFurnace returns (bool success) {\n        require(_from != address(0), "Invalid address.");\n        require(balances[_from] >= _value, "Insufficient balance.");\n        require(balances[_from] - _value >= balances[_from], "Overflow.");\n        balances[_from] = balances[_from].sub(_value);\n        totalSupply = totalSupply.sub(_value);\n        emit Transfer(_from, address(0), _value);\n        return true;\n    }\n\n    function balanceOf(address _owner) public view returns (uint256 balance) {\n        return balances[_owner];\n    }\n\n    function allowance(address _owner, address _spender) public view returns (uint256 remaining) {\n        return allowance[_owner][_spender];\n    }\n\n    function isFrozen(address _target) public view returns (bool frozen) {\n        return frozen[_target];\n    }\n\n    function isMinter(address _target) public view returns (bool isMinter) {\n        return _target == minter;\n    }\n\n    function isFurnace(address _target) public view returns (bool isFurnace) {\n        return _target == furnace;\n    }\n\n    function isFounder(address _target) public view returns (bool isFounder) {\n        return _target == founder;\n    }\n\n    function isMinterOrFurnace(address _target) public view returns (bool isMinterOrFurnace) {\n        return _target == minter || _target == furnace;\n    }\n\n    function isFounderOrMinterOrFurnace(address _target) public view returns (bool isFounderOrMinterOrFurnace) {\n        return _target == founder || _target == minter || _target == furnace;\n    }\n\n    function isFounderOrMinter(address _target) public view returns (bool isFounderOrMinter) {\n        return _target == founder || _target == minter;\n    }\n\n    function isFounderOrFurnace(address _target) public view returns (bool isFounderOrFurnace) {\n        return _target == founder || _target == furnace;\n    }\n\n    function isFounderOrMinterOrFurnace(address _target) public view returns (bool isFounderOrMinterOrFurnace) {\n        return _target == founder || _target == minter || _target == furnace;\n    }\n\n    function isFounderOrMinterOrFurnaceOrZeroAddress(address _target) public view returns (bool isFounderOrMinterOrFurnaceOrZeroAddress) {\n        return _target == founder || _target == minter || _target == furnace || _target == address(0);\n    }\n\n    function isFounderOrMinterOrFurnaceOrZeroAddressOrNullAddress(address _target) public view returns (bool isFounderOrMinterOrFurnaceOrZeroAddressOrNullAddress) {\n        return _target == founder || _target == minter || _target == furnace || _target == address(0) || _target == address(0x0);\n    }\n\n    function isFounderOrMinterOrFurnaceOrZeroAddressOrNullAddressOrAddressZero(address _target) public view returns (bool isFounderOrMinterOrFurnaceOrZeroAddressOrNullAddressOrAddressZero) {\n        return _target == founder || _target == minter || _target == furnace || _target == address(0) || _target == address(0x0) || _target == address(0x0000000000000000000000000000000000000000);\n    }\n\n    function isFounderOrMinterOrFurnaceOrZeroAddressOrNullAddressOrAddressZeroOrAddressOne(address _target) public view returns (bool isFounderOrMinterOrFurnaceOrZeroAddressOrNullAddressOrAddressZeroOrAddressOne) {\n        return _target == founder || _target == minter || _target == furnace || _target == address(0) || _target == address(0x0) || _target == address(0x0000000000000000000000000000000000000000) || _target == address(0x0000000000000000000000000000000000000001);\n    }\n\n    function isFounderOrMinterOrFurnaceOrZeroAddressOrNullAddressOrAddressZeroOrAddressOneOrAddressTwo(address _target) public view returns (bool isFounderOrMinterOrFurnaceOrZeroAddressOrNullAddressOrAddressZeroOrAddressOneOrAddressTwo) {\n        return _target == founder || _target == minter || _target == furnace || _target == address(0) || _target == address(0x0) || _target == address(0x0000000000000000000000000000000000000000) || _target == address(0x0000000000000000000000000000000000000001) || _target == address(0x0000000000000000000000000000000000000002);\n    }\n\n    function isFounderOrMinterOrFurnaceOrZeroAddressOrNullAddressOrAddressZeroOrAddressOneOrAddressTwoOrAddressThree(address _target) public view returns (bool isFounderOrMinterOrFurnaceOrZeroAddressOrNullAddressOrAddressZeroOrAddressOneOrAddressTwoOrAddressThree) {\n        return _target == founder || _target == minter || _target == furnace || _target == address(0) || _target == address(0x0) || _target == address(0x0000000000000000000000000000000000000000) || _target == address(0x0000000000000000000000000000000000000001) || _target == address(0x0000000000000000000000000000000000000002) || _target == address(0x0000000000000000000000000000000000000003);\n    }\n\n    function isFounderOrMinterOrFurnaceOrZeroAddressOrNullAddressOrAddressZeroOrAddressOneOrAddressTwoOrAddressThreeOrAddressFour(address _target) public view returns (bool isFounderOrMinterOrFurnaceOrZeroAddressOrNullAddressOrAddressZeroOrAddressOneOrAddressTwoOrAddressThreeOrAddressFour) {\n        return _target == founder || _target == minter || _target == furnace || _target == address(0) || _target == address(0x0) || _target == address(0x0000000000000000000000000000000000000000) || _target == address(0x0000000000000000000000000000000000000001) || _target == address(0x0000000000000000000000000000000000000002) || _target == address(0x0000000000000000000000000000000000000003) || _target == address(0x0000000000000000000000000000000000000004);\n    }\n\n    function isFounderOrMinterOrFurnaceOrZeroAddressOrNullAddressOrAddressZeroOrAddressOneOrAddressTwoOrAddressThreeOrAddressFourOrAddressFive(address _target) public view returns (bool isFounderOrMinterOrFurnaceOrZeroAddressOrNullAddressOrAddressZeroOrAddressOneOrAddressTwoOrAddressThreeOrAddressFourOrAddressFive) {\n        return _target == founder || _target == minter || _target == furnace || _target == address(0) || _target == address(0x0) || _target == address(0x0000000000000000000000000000000000000000) || _target == address(0x0000000000000000000000000000000000000001) || _target == address(0x0000000000000000000000000000000000000002) || _target == address(0x0000000000000000000000000000000000000003) || _target == address(0x0000000000000000000000000000000000000004) || _target == address(0x0000000000000000000000000000000000000005);\n    }\n\n    function isFounderOrMinterOrFurnaceOrZeroAddressOrNullAddressOrAddressZeroOrAddressOneOrAddressTwoOrAddressThreeOrAddressFourOrAddressFiveOrAddressSix(address _target) public view returns (bool isFounderOrMinterOrFurnaceOrZeroAddressOrNullAddressOrAddressZeroOrAddressOneOrAddressTwoOrAddressThreeOrAddressFourOrAddressFiveOrAddressSix) {\n        return _target == founder || _target == minter || _target == furnace || _target == address(0) || _target == address(0x0) || _target == address(0x0000000000000000000000000000000000000000) || _target == address(0x0000000000000000000000000000000000000001) || _target == address(0x0000000000000000000000000000000000000002) || _target == address(0x0000000000000000000000000000000000000003) || _target == address(0x0000000000000000000000000000000000000004) || _target == address(0x0000000000000000000000000000000000000005) || _target == address(0x0000000000000000000000000000000000000006);\n    }\n\n    function isFounderOrMinterOrFurnaceOrZeroAddressOrNullAddressOrAddressZeroOrAddressOneOrAddressTwoOrAddressThreeOrAddressFourOrAddressFiveOrAddressSixOrAddressSeven(address _target) public view returns (bool isFounderOrMinterOrFurnaceOrZeroAddressOrNullAddressOrAddressZeroOrAddressOneOrAddressTwoOrAddressThreeOrAddressFourOrAddressFiveOrAddressSixOrAddressSeven) {\n        return _target == founder || _target == minter || _target == furnace || _target == address(0) || _target == address(0x0) || _target == address(0x0000000000000000000000000000000000000000) || _target == address(0x0000000000000000000000000000000000000001) || _target == address(0x0000000000000000000000000000000000000002) || _target == address(0x0000000000000000000000000000000000000003) || _target == address(0x0000000000000000000000000000000000000004) || _target == address(0x0000000000000000000000000000000000000005) || _target == address(0x0000000000000000000000000000000000000006) || _target == address(0x0000000000000000000000000000000000000007);\n    }\n\n    function isFounderOrMinterOrFurnaceOrZeroAddressOrNullAddressOrAddressZeroOrAddressOneOrAddressTwoOrAddressThreeOrAddressFourOrAddressFiveOrAddressSixOrAddressSevenOrAddressEight(address _target) public view returns (bool']   """

# Actual code
actual_code = """pragma solidity 0.5.4; interface IERC20 { function balanceOf(address account) external view returns (uint256); function transfer(address recipient, uint256 amount) external returns (bool); function allowance(address owner, address spender) external view returns (uint256); function approve(address spender, uint256 amount) external returns (bool); function transferFrom(address sender, address recipient, uint256 amount) external returns (bool); event Transfer(address indexed from, address indexed to, uint256 value); event Approval(address indexed owner, address indexed spender, uint256 value); } contract Ownable { address private _owner; event OwnershipTransferred(address indexed previousOwner, address indexed newOwner); constructor() internal { _owner = msg.sender; emit OwnershipTransferred(address(0), _owner); } function owner() public view returns (address) { return _owner; } function isOwner() public view returns (bool) { return msg.sender == _owner; } modifier onlyOwner() { require(msg.sender == _owner, "Ownable: caller is not the owner"); _; } function transferOwnership(address newOwner) public onlyOwner { require(newOwner != address(0), "Ownable: new owner is the zero address"); emit OwnershipTransferred(_owner, newOwner); _owner = newOwner; } } library SafeMath { /** * @dev Returns the addition of two unsigned integers, reverting on * overflow. * * Counterpart to Solidity's `+` operator. * * Requirements: * - Addition cannot overflow. */ function add(uint256 a, uint256 b) internal pure returns (uint256) { uint256 c = a + b; require(c >= a, "SafeMath: addition overflow"); return c; } /** * @dev Returns the subtraction of two unsigned integers, reverting on * overflow (when the result is negative). * * Counterpart to Solidity's `-` operator. * * Requirements: * - Subtraction cannot overflow. */ function sub(uint256 a, uint256 b) internal pure returns (uint256) { return sub(a, b, "SafeMath: subtraction overflow"); } /** * @dev Returns the subtraction of two unsigned integers, reverting with custom message on * overflow (when the result is negative). * * Counterpart to Solidity's `-` operator. * * Requirements: * - Subtraction cannot overflow. * * _Available since v2.4.0._ */ function sub(uint256 a, uint256 b, string memory errorMessage) internal pure returns (uint256) { require(b <= a, errorMessage); uint256 c = a - b; return c; } /** * @dev Returns the multiplication of two unsigned integers, reverting on * overflow. * * Counterpart to Solidity's `*` operator. * * Requirements: * - Multiplication cannot overflow. */ function mul(uint256 a, uint256 b) internal pure returns (uint256) { // Gas optimization: this is cheaper than requiring 'a' not being zero, but the // benefit is lost if 'b' is also tested. // See: https://github.com/OpenZeppelin/openzeppelin-contracts/pull/522 if (a == 0) { return 0; } uint256 c = a * b; require(c / a == b, "SafeMath: multiplication overflow"); return c; } /** * @dev Returns the integer division of two unsigned integers. Reverts on * division by zero. The result is rounded towards zero. * * Counterpart to Solidity's `/` operator. Note: this function uses a * `revert` opcode (which leaves remaining gas untouched) while Solidity * uses an invalid opcode to revert (consuming all remaining gas). * * Requirements: * - The divisor cannot be zero. */ function div(uint256 a, uint256 b) internal pure returns (uint256) { return div(a, b, "SafeMath: division by zero"); } /** * @dev Returns the integer division of two unsigned integers. Reverts with custom message on * division by zero. The result is rounded towards zero. * * Counterpart to Solidity's `/` operator. Note: this function uses a * `revert` opcode (which leaves remaining gas untouched) while Solidity * uses an invalid opcode to revert (consuming all remaining gas). * * Requirements: * - The divisor cannot be zero. * * _Available since v2.4.0._ */ function div(uint256 a, uint256 b, string memory errorMessage) internal pure returns (uint256) { // Solidity only automatically asserts when dividing by 0 require(b > 0, errorMessage); uint256 c = a / b; // assert(a == b * c + a % b); // There is no case in which this doesn't hold return c; } /** * @dev Returns the remainder of dividing two unsigned integers. (unsigned integer modulo), * Reverts when dividing by zero. * * Counterpart to Solidity's `%` operator. This function uses a `revert` * opcode (which leaves remaining gas untouched) while Solidity uses an * invalid opcode to revert (consuming all remaining gas). * * Requirements: * - The divisor cannot be zero. */ function mod(uint256 a, uint256 b) internal pure returns (uint256) { return mod(a, b, "SafeMath: modulo by zero"); } /** * @dev Returns the remainder of dividing two unsigned integers. (unsigned integer modulo), * Reverts with custom message when dividing by zero. * * Counterpart to Solidity's `%` operator. This function uses a `revert` * opcode (which leaves remaining gas untouched) while Solidity uses an * invalid opcode to revert (consuming all remaining gas). * * Requirements: * - The divisor cannot be zero. * * _Available since v2.4.0._ */ function mod(uint256 a, uint256 b, string memory errorMessage) internal pure returns (uint256) { require(b != 0, errorMessage); return a % b; } } contract ZildFinanceCoin is Ownable, IERC20 { using SafeMath for uint256; string public constant name = 'Zild Finance Coin'; string public constant symbol = 'Zild'; uint8 public constant decimals = 18; uint256 public totalSupply = 9980 * 10000 * 10 ** uint256(decimals); uint256 public allowBurn = 2100 * 10000 * 10 ** uint256(decimals); uint256 public tokenDestroyed; uint256 public constant FounderAllocation = 1497 * 10000 * 10 ** uint256(decimals); uint256 public constant FounderLockupAmount = 998 * 10000 * 10 ** uint256(decimals); uint256 public constant FounderLockupCliff = 365 days; uint256 public constant FounderReleaseInterval = 30 days; uint256 public constant FounderReleaseAmount = 20.7916 * 10000 * 10 ** uint256(decimals); uint256 public constant MarketingAllocation = 349 * 10000 * 10 ** uint256(decimals); uint256 public constant FurnaceAllocation = 150 * 10000 * 10 ** uint256(decimals); address public founder = address(0); uint256 public founderLockupStartTime = 0; uint256 public founderReleasedAmount = 0; mapping (address => uint256) private _balances; mapping (address => mapping (address => uint256)) private _allowances; mapping (address => bool) public frozenAccount; event Transfer(address indexed from, address indexed to, uint256 value); event Approval(address indexed from, address indexed to, uint256 value); event ChangeFounder(address indexed previousFounder, address indexed newFounder); event SetMinter(address indexed minter); event SetMarketing(address indexed marketing); event SetFurnace(address indexed furnace); event Burn(address indexed _from, uint256 _tokenDestroyed, uint256 _timestamp); event FrozenFunds(address target, bool frozen); constructor(address _founder, address _marketing) public { require(_founder != address(0), "ZildFinanceCoin: founder is the zero address"); require(_marketing != address(0), "ZildFinanceCoin: operator is the zero address"); founder = _founder; founderLockupStartTime = block.timestamp; _balances[address(this)] = totalSupply; _transfer(address(this), _marketing, MarketingAllocation); } function release() public { uint256 currentTime = block.timestamp; uint256 cliffTime = founderLockupStartTime.add(FounderLockupCliff); if (currentTime < cliffTime) return; if (founderReleasedAmount >= FounderLockupAmount) return; uint256 month = currentTime.sub(cliffTime).div(FounderReleaseInterval); uint256 releaseAmount = month.mul(FounderReleaseAmount); if (releaseAmount > FounderLockupAmount) releaseAmount = FounderLockupAmount; if (releaseAmount <= founderReleasedAmount) return; uint256 amount = releaseAmount.sub(founderReleasedAmount); founderReleasedAmount = releaseAmount; _transfer(address(this), founder, amount); } function balanceOf(address account) public view returns (uint256) { return _balances[account]; } function transfer(address to, uint256 amount) public returns (bool) { require(to != address(0), "ERC20: tranfer to the zero address"); require(!frozenAccount[msg.sender]); require(!frozenAccount[to]); _transfer(msg.sender, to, amount); return true; } function burn(uint256 _value) public returns (bool){ _burn(msg.sender, _value); return true; } function _burn(address _who, uint256 _burntAmount) internal { require (tokenDestroyed.add(_burntAmount) <= allowBurn, "ZildFinanceCoin: exceeded the maximum allowable burning amount" ); require(_balances[msg.sender] >= _burntAmount && _burntAmount > 0); _transfer(address(_who), address(0), _burntAmount); totalSupply = totalSupply.sub(_burntAmount); tokenDestroyed = tokenDestroyed.add(_burntAmount); emit Burn(_who, _burntAmount, block.timestamp); } function allowance(address from, address to) public view returns (uint256) { return _allowances[from][to]; } function approve(address to, uint256 amount) public returns (bool) { _approve(msg.sender, to, amount); return true; } function transferFrom(address from, address to, uint256 amount) public returns (bool) { uint256 remaining = _allowances[from][msg.sender].sub(amount, "ERC20: transfer amount exceeds allowance"); require(to != address(0), "ERC20: tranfer to the zero address"); require(!frozenAccount[from]); require(!frozenAccount[to]); require(!frozenAccount[msg.sender]); _transfer(from, to, amount); _approve(from, msg.sender, remaining); return true; } function _transfer(address from, address to, uint256 amount) private { require(from != address(0), "ERC20: transfer from the zero address"); _balances[from] = _balances[from].sub(amount, "ERC20: transfer amount exceeds balance"); _balances[to] = _balances[to].add(amount); emit Transfer(from, to, amount); } function _approve(address from, address to, uint256 amount) private { require(from != address(0), "ERC20: approve from the zero address"); require(to != address(0), "ERC20: approve to the zero address"); _allowances[from][to] = amount; emit Approval(from, to, amount); } function changeFounder(address _founder) public onlyOwner { require(_founder != address(0), "ZildFinanceCoin: founder is the zero address"); emit ChangeFounder(founder, _founder); founder = _founder; } function setMinter(address minter) public onlyOwner { require(minter != address(0), "ZildFinanceCoin: minter is the zero address"); require(_balances[minter] == 0, "ZildFinanceCoin: minter has been initialized"); _transfer(address(this), minter, totalSupply.sub(FounderAllocation)); emit SetMinter(minter); } function setFurnace(address furnace) public onlyOwner { require(furnace != address(0), "ZildFinanceCoin: furnace is the zero address"); require(_balances[furnace] == 0, "ZildFinanceCoin: furnace has been initialized"); _transfer(address(this), furnace, FurnaceAllocation); emit SetFurnace(furnace); } function freezeAccount(address _target, bool _bool) public onlyOwner { if (_target != address(0)) { frozenAccount[_target] = _bool; emit FrozenFunds(_target,_bool); } } }"""

# Tokenize the generated and actual code
gen_tokens = generated_code.split()
act_tokens = actual_code.split()

# Calculate BLEU score
bleu_score = sentence_bleu([act_tokens], gen_tokens)
print("BLEU Score:", bleu_score)

BLEU Score: 0.06089870750884132